In [23]:
%%writefile app.py
import streamlit as st
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
from openai import OpenAI

# OpenAI client (API key must be set in environment)
client = OpenAI()

st.set_page_config(page_title="RAG PDF QA", layout="centered")
st.title("📄 RAG-based Document Q&A")

@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

uploaded_file = st.file_uploader("Upload a PDF", type=["pdf"])

if uploaded_file:
    # Read PDF
    reader = PdfReader(uploaded_file)
    text = ""
    for page in reader.pages:
        if page.extract_text():
            text += page.extract_text()

    # Chunk text
    def chunk_text(text, chunk_size=800, overlap=50):
        chunks = []
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunks.append(text[start:end])
            start = end - overlap
        return chunks

    chunks = chunk_text(text)

    # Create embeddings
    embeddings = embedder.encode(chunks, convert_to_numpy=True)

    # Build FAISS index
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    st.success("Document indexed successfully!")

    # Ask question
    query = st.text_input("Ask a question")

    if query:
        query_emb = embedder.encode([query], convert_to_numpy=True)
        _, indices = index.search(query_emb, k=6)

        # Retrieve context
        context = " ".join([chunks[i] for i in indices[0]])

        # LLM prompt
        prompt = f"""
You are a helpful assistant.

Answer the question using the context below.
You may paraphrase or summarize, but do not add new facts.
If the context is insufficient, say:
"Not enough information in the document."

Context:
{context}

Question:
{query}
"""

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        st.subheader("✅ Answer")
        st.write(response.choices[0].message.content)

        with st.expander("Retrieved Context"):
            st.write(context)


Overwriting app.py


In [27]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared


In [28]:
!streamlit run app.py &>/content/logs.txt &

In [29]:
!cloudflared tunnel --url http://localhost:8501

2026-01-17T05:32:42Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-17T05:32:42Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-17T05:32:45Z INF +--------------------------------------------------------------------------------------------+
2026-01-17T05:32:45Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-01-17T05:32:45Z INF |  https://newly-pubmed-mutual-carroll.trycloudflare.com